<a href="https://colab.research.google.com/github/Jondoloh/Bootcamp_Project_6/blob/main/Project_6_Bootcamp_Finale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Project 6 : Toxic Comments Classification

                                by 

                        1. John Ndolo
                        2. Albert Agisha
                        3. Yvan Pimi
                        4. Mohammed Abukalam 

Toxic Comment Classification using Logistic Regression

### 1. Installing dependencies

In [ ]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
import pandas
#{"username":"yvanpimi","key":"af92ea4ea3c8ef09193026c5624eb63d"}

od.download(
    "https://www.kaggle.com/datasets/julian3833/jigsaw-toxic-comment-classification-challenge")


Skipping, found downloaded files in "./jigsaw-toxic-comment-classification-challenge" (use force=True to force download)


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
#import contractions
import string

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2. Loading Dataset

In [ ]:
#loading dataset
train = pd.read_csv('/content/jigsaw-toxic-comment-classification-challenge/train.csv')
train.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
test = pd.read_csv('/content/jigsaw-toxic-comment-classification-challenge/test.csv')
test.head(10)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [ ]:
test_labels = pd.read_csv('/content/jigsaw-toxic-comment-classification-challenge/test_labels.csv')
#test_labels.head()

In [ ]:
samp_sub = pd.read_csv('/content/jigsaw-toxic-comment-classification-challenge/sample_submission.csv')
#samp_sub.head()

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [ ]:
train.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [ ]:
# categories = train['toxic'].unique().tolist()
# categories

In [ ]:
# Let's check some comments
for i in range(10):
    print(train['comment_text'][i])
    print('*'*20)

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
********************
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
********************
Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
********************
"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you h

### 3. Data pre-processing

In [ ]:
def to_lowercase(text):
    return text.lower()

# Remove website links
def remove_links(text):
    template = re.compile(r'https?://\S+|www\.\S+') 
    text = template.sub(r'', text)
    return text

# Remove HTML tags
def remove_html(text):
    template = re.compile(r'<[^>]*>') 
    text = template.sub(r'', text)
    return text


# Remove stopwords
def remove_stopwords(words, stop_words):
    return [word for word in words if word not in stop_words]

# Remove none ascii characters
def remove_non_ascii(text):
    template = re.compile(r'[^\x00-\x7E]+') 
    text = template.sub(r'', text)
    return text

# Replace none printable characters
def remove_non_printable(text):
    template = re.compile(r'[\x00-\x0F]+') 
    text = template.sub(r' ', text)
    return text

# Remove special characters
def remove_special_chars(text):
        text = re.sub("'s", '', text)
        template = re.compile('["#$%&\'()\*\+-/:;<=>@\[\]\\\\^_`{|}~]') 
        text = template.sub(r' ', text)
        return text

# Replace multiple punctuation 
def replace_multiplt_punc(text):
        text = re.sub('[.!?]{2,}', '.', text)
        text = re.sub(',+', ',', text) 
        return text

    # Remove numbers
def remove_numbers(text):
        text = re.sub('\d+', ' ', text)
        return text

def handle_spaces(text):
    # Remove extra spaces
    text = re.sub('\s+', ' ', text)
    
    # Remove spaces at the beginning and at the end of string
    text = text.strip() 
    
    return text

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# def stem_words(words):
#     """Stem words in text"""
#     stemmer = PorterStemmer()
#     return [stemmer.stem(word) for word in words]

def text2words(text):
      return word_tokenize(text)
    
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ([lemmatizer.lemmatize(word, pos='v') for word in words])

def remove_pattern(text): 
    # remove hi moron 
    text= re.sub(r'(hi)(.*)\1', r'\1', text)
    # remove duplicate words
    text= re.sub(r"\b(\w+)(?:\W+\1\b)+",r'\1', text,flags=re.IGNORECASE)
    # remove [User:Cirt]] 
    text= re.sub(r"\[.*?\]", ' ', text)
    # remove \n\n
    text= re.sub(r"\n", ' ', text)
    return text

In [ ]:
def pre_process( text):
    #text = remove_contractions(text)
    text = remove_pattern(text)
    text = remove_links(text)
    text = remove_html(text)
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_non_printable(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = handle_spaces(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words) #either stem or lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ' '.join(words)

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda x: pre_process(x))

In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edit make username hardcore metall...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww match background colour seemingly stick th...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really try edit war guy constantly rem...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggestion improvement wonder sectio...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0


In [ ]:
test['comment_text'] = test['comment_text'].apply(lambda x: pre_process(x))
test.head()

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule succesful ever whats hat sad ...
1,0000247867823ef7,rfc title fine imo
2,00013b17ad220c46,source zawe ashton lapland
3,00017563c3f7919a,look back source information update correct fo...
4,00017695ad8997eb,anonymously edit article


### 4.Models

In [ ]:
x_train = train['comment_text']
x_train.head()

0    explanation edit make username hardcore metall...
1    aww match background colour seemingly stick th...
2    hey man really try edit war guy constantly rem...
3    make real suggestion improvement wonder sectio...
4                        sir hero chance remember page
Name: comment_text, dtype: object

In [ ]:
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [ ]:
x_test = test['comment_text']
x_test.head()

0    yo bitch ja rule succesful ever whats hat sad ...
1                                   rfc title fine imo
2                           source zawe ashton lapland
3    look back source information update correct fo...
4                             anonymously edit article
Name: comment_text, dtype: object

In [ ]:
samp_sub

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.5,0.5,0.5,0.5,0.5,0.5
153160,fffd7a9a6eb32c16,0.5,0.5,0.5,0.5,0.5,0.5
153161,fffda9e8d6fafa9e,0.5,0.5,0.5,0.5,0.5,0.5
153162,fffe8f1340a79fc2,0.5,0.5,0.5,0.5,0.5,0.5


In [ ]:
# library imported 
import numpy as np
import pandas as pd
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer 
import copy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import numpy as np 
from scipy.special import logsumexp

class MNNaiveBayes:
    """
    This class is written with the standard python library and NumPy.
    The goal is to recreate a multinomial Naive Bayes classifcation model from scratch.
    
    Usage:
    1. Instantiate the model
    2. Fit it on a corpus of documents and an array of their binary classifications
    3. Predict on another corpus to get an array of binary classes
    """
    def __init__(self, k=0.5):
        self.k = k
        self.cat0_count = 0
        self.cat1_count = 0
        self.total_count = self.cat0_count + self.cat1_count
        self.cat_0_prior = 0
        self.cat_1_prior = 0
        self.cat_0_prior, self.cat_1_prior
        self.word_probs = []
        self.vocab = []

    def tokenize(self, document):
        """
        Take in a document and return a list of words
        """
        doc = document.lower()
        # remove non-alpha characters
        stop_chars = '''0123456789!()-[]{};:'"\,<>./?@#$%^&*_~'''
        
        tokens = ""
        # iterate through and make each token
        for char in doc:
            if char not in stop_chars:
                tokens += char

        return tokens.split() # now a list of tokens
  
    def count_words(self, X, y):
        """
        X is an array of documents
        y is an array of targets, 0 or 1
        Output a dictionary of {word: (cat0_count, cat1_count)...}
        """
        counts = {}
        # need to figure our this loop, want to iterate over both of them, I see why it was paired before
        for document, category in zip(X, y):
            for token in self.tokenize(document):
                # Initialize a dict entry with 0 counts
                if token not in counts:
                    counts[token] = [0,0]
                # Now that it exists, add to the category count for that word
                counts[token][category] += 1
        return counts

    def prior_prob(self, counts):
        
        # Iterate through counts dict and add up each word count by category
        cat0_word_count = cat1_word_count = 0
        for word, (cat0_count, cat1_count) in counts.items():
            cat0_word_count += cat0_count
            cat1_word_count += cat1_count

        # save attributes to the class
        self.cat0_count = cat0_word_count
        self.cat1_count = cat1_word_count
        self.total_count = self.cat0_count + self.cat1_count

        # Get the prior prob by dividing words in each cat by total words
        cat_0_prior = cat0_word_count / self.total_count
        cat_1_prior = cat1_word_count / self.total_count
        return cat_0_prior, cat_1_prior

    def word_probabilities(self, counts):
        """turn the word_counts into a list of triplets
        word, p(w | cat0), and p(w | cat1)"""
        # Here we apply the smoothing term, self.k, so that words that aren't in
        # the category don't get calculated as 0
        self.vocab = [word for word, (cat0, cat1) in counts.items()]
        word_probs = {}
        for word, (cat0, cat1) in counts.items():
            word_probs[word] = ((cat0 + self.k) / (self.cat0_count + 2 * self.k), (cat1 + self.k) / (self.cat1_count + 2 * self.k))
        # return [(word,
        # (cat0 + self.k) / (self.cat0_count + 2 * self.k),
        # (cat1 + self.k) / (self.cat1_count + 2 * self.k))
        # for word, (cat0, cat1) in counts.items()]
        return word_probs

    def fit(self, X, y):
        # Take all these functions and establish probabilities of input
        counts = self.count_words(X, y)
        self.cat_0_prior, self.cat_1_prior = self.prior_prob(counts)
        self.word_probs = self.word_probabilities(counts)

    def predict(self, test_corpus):
    # Split the text into tokens,
    # For each category: calculate the probability of each word in that cat
    # find the product of all of them and the prior prob of that cat
        y_pred = []
        y_pred_proba = []
        y_cat0 = []
        y_cat1 = []
        joint_log_likelihood = []
        for document in test_corpus:
            # Every document get their own prediction probability
            log_prob_cat0 = log_prob_cat1 = 0.0
            tokens = self.tokenize(document)
                # Iterate through the training vocabulary and add any log probs that match
                # if no match don't do anything. We just need a score for each category/doc
            for word in tokens:
                if word in self.word_probs:
                    prob_cat0 = self.word_probs[word][0]
                    prob_cat1 = self.word_probs[word][0]

                    log_prob_cat0 += np.log(prob_cat0)
                    log_prob_cat1 += np.log(prob_cat1)            
            # for word, prob_cat0, prob_cat1 in self.word_probs:
            #     if word in tokens:
            #     # Because of 'overflow' best to add the log probs together and exp
            #         log_prob_cat0 += np.log(prob_cat0)
            #         log_prob_cat1 += np.log(prob_cat1)

                # get each of the category predictions including the prior
            # cat_0_pred = self.cat_0_prior * np.exp(log_prob_cat0)
            # cat_1_pred = self.cat_1_prior * np.exp(log_prob_cat1)

            cat_0_pred = np.log(self.cat_0_prior) + log_prob_cat0
            cat_1_pred = np.log(self.cat_1_prior) + log_prob_cat1
            
            y_cat0.append(cat_0_pred)
            y_cat1.append(cat_1_pred)

            if cat_0_pred >= cat_1_pred:
                y_pred.append(0)
                y_pred_proba.append(cat_0_pred)
            else:
                y_pred.append(1)
                y_pred_proba.append(cat_1_pred)
        y_pred = np.array(y_pred)

        # joint_log_likelihood.append(y_cat0)
        # joint_log_likelihood.append(y_cat1)
        # jll = np.array(joint_log_likelihood).T
        # log_prob_x = logsumexp(jll, axis=1)
        # y_pred_proba = np.array(y_pred_proba).reshape(-1, 1)

        # log_prob_x = logsumexp(y_pred_proba, axis=1)
        # self.predict_proba = np.exp(jll - np.atleast_2d(log_prob_x).T) 
        self.predict_proba = np.exp(y_pred_proba) 
        # self.predict_proba = y_pred_proba 
        return y_pred


In [ ]:
transformer = MNNaiveBayes() #init model
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
# logreg = LogisticRegression(C=1.0, solver='liblinear')

# create submission file
df_submission = samp_sub
labels =[
    'toxic', 
    'severe_toxic', 'obscene', 'threat','insult', 'identity_hate'
]
# x_train = x_train.to_numpy().reshape(-1, 1)
for label in labels:
    print('> Processing ' + label)
    y = train[label]
    # print(type(y))
    # train the model using X_dtm & y
    # transformer.fit(x_train, y)
    transformer.fit(x_train, y)
    # compute the training accuracy
    y_pred_X = transformer.predict(x_test)
    # print(len(y_pred_X), len(y[:len(y_pred_X)]))
    print('Training accuracy is ' + str(accuracy_score(y[:len(y_pred_X)], y_pred_X)))
    # print('Training accuracy is ' + str(transformer.accuracy(y[:len(y_pred_X)], y_pred_X)))
    print('-'*10)

    # compute the predicted probabilities for X_test_dtm
    test_y_prob = transformer.predict_proba
    # print(test_y_prob[0])
    df_submission[label] = test_y_prob

> Processing toxic
Training accuracy is 0.904200726019169
----------
> Processing severe_toxic
Training accuracy is 0.9899323600846152
----------
> Processing obscene
Training accuracy is 0.9469131127418976
----------
> Processing threat
Training accuracy is 0.9970358569898932
----------
> Processing insult
Training accuracy is 0.9505627954349586
----------
> Processing identity_hate
Training accuracy is 0.991225092058186
----------


### 5. Submission file

In [ ]:
df_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-149.177303,-141.860518,-147.610716,-139.842251,-146.362691,-140.338324
1,0000247867823ef7,-33.300136,-33.387706,-33.342536,-33.424115,-33.345736,-33.410384
2,00013b17ad220c46,-32.805341,-32.918677,-32.868043,-32.944806,-32.872819,-32.933930
3,00017563c3f7919a,-101.699347,-101.896389,-101.769340,-102.044713,-101.785828,-101.985028
4,00017695ad8997eb,-20.321250,-20.382998,-20.358585,-20.405879,-20.359319,-20.396278


In [ ]:
df_submission.to_csv('submission.csv',index=False)

**<h1>Logistic Regression</h1>**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_vec_train = vectorizer.fit_transform(x_train)
X_vec_test = vectorizer.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
logreg = LogisticRegression(C=1.0, solver='liblinear')

# create submission file
df_submission = samp_sub
labels =['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
for label in labels:
    print('> Processing ' + label)
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_vec_train, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_vec_test)
    print('Training accuracy is ' + str(accuracy_score(y[:len(y_pred_X)], y_pred_X)))
    print('-'*10)
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(X_vec_test)[:,1]
    df_submission[label] = test_y_prob

> Processing toxic
Training accuracy is 0.7702985035648063
----------
> Processing severe_toxic
Training accuracy is 0.9821955550912748
----------
> Processing obscene
Training accuracy is 0.8653926510145987
----------
> Processing threat
Training accuracy is 0.9958475882061059
----------
> Processing insult
Training accuracy is 0.88789141051422
----------
> Processing identity_hate
Training accuracy is 0.9846634979499099
----------


In [ ]:
df_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998637,0.018101,0.997131,0.014218,0.927242,0.100646
1,0000247867823ef7,0.000512,0.000340,0.000218,0.000410,0.000771,0.000507
2,00013b17ad220c46,0.002407,0.000266,0.001424,0.000006,0.001479,0.000084
3,00017563c3f7919a,0.000966,0.000916,0.000687,0.000074,0.001082,0.000024
4,00017695ad8997eb,0.001941,0.000570,0.001445,0.000212,0.001741,0.000451


In [ ]:
df_submission.to_csv('submission_logistic.csv',index=False)

**<h1>Multinomial Naives Bayes</h1>**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
# model = MultinomialNB().fit(title_tfidf, df['CATEGORY'])
model = MultinomialNB(alpha=2.0)
df_submission = samp_sub

labels =['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
for label in labels:
    print('> Processing ' + label)
    y = train[label]
    # train the model using X_dtm & y
    model.fit(X_vec_train, y)
    # compute the training accuracy
    y_pred_X = model.predict(X_vec_test)
    print('Training accuracy is ' + str(accuracy_score(y[:len(y_pred_X)], y_pred_X)))
    print('-'*10)
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = model.predict_proba(X_vec_test)[:,1]
    df_submission[label] = test_y_prob

> Processing toxic
Training accuracy is 0.8110260896816485
----------
> Processing severe_toxic
Training accuracy is 0.9896320284139877
----------
> Processing obscene
Training accuracy is 0.8971886344049516
----------
> Processing threat
Training accuracy is 0.9970227990911702
----------
> Processing insult
Training accuracy is 0.91340654461884
----------
> Processing identity_hate
Training accuracy is 0.9911075709696796
----------


In [ ]:
df_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.935429,0.035778,0.821750,0.002060,0.725726,0.025249
1,0000247867823ef7,0.012085,0.000831,0.005530,0.000295,0.005069,0.000867
2,00013b17ad220c46,0.022863,0.000430,0.008986,0.000078,0.007528,0.000392
3,00017563c3f7919a,0.005655,0.000049,0.001583,0.000008,0.001506,0.000034
4,00017695ad8997eb,0.051979,0.004747,0.034621,0.001187,0.031907,0.004153


In [ ]:
df_submission.to_csv('submission_multinomial.csv',index=False)

**<h1>Bernouilli Bayes</h1>**

In [ ]:

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, f1_score
# model = MultinomialNB().fit(title_tfidf, df['CATEGORY'])
model = BernoulliNB()
df_submission = samp_sub

labels =['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
for label in labels:
    print('> Processing ' + label)
    y = train[label]
    # train the model using X_dtm & y
    model.fit(X_vec_train, y)
    # compute the training accuracy
    y_pred_X = model.predict(X_vec_test)
    print('Training accuracy is ' + str(accuracy_score(y[:len(y_pred_X)], y_pred_X)))
    print('-'*10)
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = model.predict_proba(X_vec_test)[:,1]
    df_submission[label] = test_y_prob

> Processing toxic
Training accuracy is 0.5801624402601133
----------
> Processing severe_toxic
Training accuracy is 0.9067666031182262
----------
> Processing obscene
Training accuracy is 0.6573019769658667
----------
> Processing threat
Training accuracy is 0.9852249875949962
----------
> Processing insult
Training accuracy is 0.6669778799195634
----------
> Processing identity_hate
Training accuracy is 0.9394505236217388
----------


In [ ]:
df_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1.000000,9.971238e-01,0.999998,4.353934e-03,0.999995,9.988326e-01
1,0000247867823ef7,0.092700,1.344439e-04,0.015078,6.808949e-11,0.014236,1.911128e-05
2,00013b17ad220c46,0.798373,2.747946e-03,0.607348,2.103857e-11,0.569380,2.143707e-04
3,00017563c3f7919a,0.001492,9.296733e-08,0.000277,2.972793e-14,0.000283,1.863902e-08
4,00017695ad8997eb,0.304350,1.970560e-03,0.236163,2.862754e-10,0.214981,2.163735e-04


In [ ]:
df_submission.to_csv('submission_bernoulli.csv',index=False)